In [30]:
import os
import pandas as pd
import glob

#1.read in and combine all the datasets

csv_files = glob.glob("*.csv")

df_list = []

for file in csv_files:
    name = os.path.splitext(os.path.basename(file))[0]
    df = pd.read_csv(file)
    df = df.iloc[:,:-1]
    
    df_melted = pd.melt(
        df,
        id_vars=['Period'],
        var_name='Year',
        value_name='Area_Percentage'
    )


    df_melted['County']=name

    df_list.append(df_melted)

combined_df = pd.concat(df_list,ignore_index=True)
combined_df['Year'] = combined_df['Year'].str[:4]
combined_df['Year'] = combined_df['Year'].astype(int)
combined_df.rename(columns={"Period": "LC_Types"}, inplace=True)

Combined_NLCD = pd.DataFrame(combined_df) 
Combined_NLCD.to_csv(os.path.join("./Processed", "Combined_NLCD_percentage.csv"))


In [2]:
import os
import pandas as pd
import glob
Combined_NLCD = pd.read_csv("./Processed/Combined_NLCD_percentage.csv")
Combined_NLCD.dropna(subset=["Area_Percentage"], inplace=True)
# Combined_NLCD["Year"] = pd.to_datetime(Combined_NLCD["Year"], format="%Y")

In [3]:
type_map = {
    "developed high intensity": "developed sites",
    "developed low intensity": "developed sites",
    "developed medium intensity": "developed sites",
    "developed open space": "developed sites",

    "deciduous forest": "deciduous forest",
    "evergreen forest": "evergreen forest",
    "mixed forest": "mixed forest",

    "emergent herbaceous wetlands": "wetland",
    "woody wetlands": "wetland",

    "open water": "open water",
    "barren land (rock/sand/clay)": "barren land (rock/sand/clay)",
    "cultivated crops": "cultivated crops",
    "grasslands/herbaceous": "grasslands/herbaceous",
    "pasture/hay": "pasture/hay",
    "shrub/shrub": "shrub/shrub"
    }


Combined_NLCD["LC_Grouped"] = Combined_NLCD["LC_Types"].str.lower().str.strip().map(type_map)

early = Combined_NLCD[(Combined_NLCD["Year"] >= 2005) & (Combined_NLCD["Year"] <= 2010)]
late = Combined_NLCD[(Combined_NLCD["Year"] >= 2018) & (Combined_NLCD["Year"] <= 2023)]

print(early.head())
print(late.head())
early_avg = early.groupby(["County", "LC_Grouped"])["Area_Percentage"].mean().reset_index().rename(columns={"Area_Percentage": "Early_Avg"})
late_avg = late.groupby(["County", "LC_Grouped"])["Area_Percentage"].mean().reset_index().rename(columns={"Area_Percentage": "Late_Avg"})

merged = pd.merge(early_avg, late_avg, on=["County", "LC_Grouped"], how="outer")

merged["Change_Rate"] = ((merged["Late_Avg"] - merged["Early_Avg"]) / merged["Early_Avg"])*100

merged.to_csv(os.path.join("./Processed", "Change_Percentage_05to10.csv"), index=False)
 


     Unnamed: 0                    LC_Types  Year  Area_Percentage  \
300         300                  Open Water  2005         0.721094   
301         301        Developed Open Space  2005         5.185775   
302         302     Developed Low Intensity  2005         1.932031   
303         303  Developed Medium Intensity  2005         0.262457   
304         304    Developed High Intensity  2005         0.099447   

             County       LC_Grouped  
300  Appling County       open water  
301  Appling County  developed sites  
302  Appling County  developed sites  
303  Appling County  developed sites  
304  Appling County  developed sites  
     Unnamed: 0                    LC_Types  Year  Area_Percentage  \
495         495                  Open Water  2018         0.774549   
496         496        Developed Open Space  2018         5.041421   
497         497     Developed Low Intensity  2018         2.194827   
498         498  Developed Medium Intensity  2018         0.37323

In [ ]:
import pandas as pd
Combined_NLCD = pd.read_csv("/mnt/data/Combined_NLCD_percentage.csv")
developed = [
    "Developed Open Space",
    "Developed Low Intensity",
    "Developed Medium Intensity",
    "Developed High Intensity"
]
forest = [
    "Deciduous Forest",
    "Evergreen Forest",
    "Mixed Forest"
]
wetland = [
    "Woody Wetlands",
    "Emergent Herbaceous Wetlands"
]

def assign_group(land_type):
    if land_type in developed:
        return "Developed"
    elif land_type in forest:
        return "Forest"
    elif land_type in wetland:
        return "Wetland"
    else:
        return None


df["LC_Grouped"] = df["LC_Types"].apply(assign_group)
early = df[(df["Year"] >= 2000) & (df["Year"] <= 2005)]
late = df[(df["Year"] >= 2018) & (df["Year"] <= 2023)]

df_grouped = df.dropna(subset=["LC_Grouped"]).copy()
df_grouped = df_grouped.groupby(["County", "Year", "LC_Grouped"])["Area_Percentage"].sum().reset_index()
df_grouped.rename(columns={"LC_Grouped": "LC_Types"}, inplace=True)

combined = pd.concat([df[["County", "Year", "LC_Types", "Area_Percentage"]], df_grouped], ignore_index=True)
early_combined = combined[(combined["Year"] >= 2000) & (combined["Year"] <= 2025)]
late_combined = combined[(combined["Year"] >= 2018) & (combined["Year"] <= 2023)]

early_avg = early_combined.groupby(["County", "LC_Types"])["Area_Percentage"].mean().reset_index().rename(columns={"Area_Percentage": "Early_Avg"})
late_avg = late_combined.groupby(["County", "LC_Types"])["Area_Percentage"].mean().reset_index().rename(columns={"Area_Percentage": "Late_Avg"})


merged = pd.merge(early_avg, late_avg, on=["County", "LC_Types"], how="outer")
merged["Change_Rate"] = ((merged["Late_Avg"] - merged["Early_Avg"]) /
                            merged["Early_Avg"].replace(0, pd.NA)) * 100
